In [1]:
import sys
import networkx as nx
import matplotlib.pyplot as plt
import pygraphviz as pgv
import random
from os import listdir
import pickle
import time
import instance
import statistics
from utils import read_param,add_lists,sub_lists, less_than, min_finish_time, find_index #Utility functions
import numpy as np
from deap import base,creator,tools,algorithms,gp
import operator
import math

In [2]:
train_set=[]

types=['j30','j60']
for typ in types:
  for i in range(1,49):
    train_set.append("./"+typ+'/'+typ+str(i)+"_1.sm")
    train_set.append("./"+typ+'/'+typ+str(i)+"_2.sm")


In [3]:

POP_SIZE=1024
NUM_GENERATIONS=25
INST_TYPE='j60'
MATING_PROB=0.5
MUTATION_PROB=0.3
SELECTION_POOL_SIZE=7
HOF_SIZE=1
HEIGHT_LIMIT = 6
MU=1024
LAMBDA=1024
GEN_MIN_HEIGHT=3
GEN_MAX_HEIGHT=5

In [4]:
def div(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1


pset = gp.PrimitiveSet("MAIN",10)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(div, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(max, 2)
pset.addPrimitive(min, 2)
pset.renameArguments(ARG0="ES")
pset.renameArguments(ARG1="EF")
pset.renameArguments(ARG2="LS")
pset.renameArguments(ARG3="LF")
pset.renameArguments(ARG4="TPC")
pset.renameArguments(ARG5="TSC")
pset.renameArguments(ARG6="RR")
pset.renameArguments(ARG7="AvgRReq")
pset.renameArguments(ARG8="MaxRReq")
pset.renameArguments(ARG9="MinRReq")
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)
counts=0
def evalSymbReg(individual):
    # Transform the tree expression in a callable function
    
    func = toolbox.compile(expr=individual)
    sumv=0
    for i in range(len(train_set)):
        
        file=train_set[i]
        
        inst=instance.instance(file,use_precomputed=True)
        priorities=[0]*(inst.n_jobs+1)
        for j in range(1,inst.n_jobs+1):
            priorities[j]=func(inst.earliest_start_times[j],inst.earliest_finish_times[j],inst.latest_start_times[j],inst.latest_finish_times[j],inst.mtp[j],inst.mts[j],inst.rr[j],inst.avg_rreq[j],inst.max_rreq[j],inst.min_rreq[j])

        frac,makespan=inst.parallel_sgs(option='forward',priority_rule='',priorities=priorities)
        sumv+=frac
    
    return (sumv/len(train_set),)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=GEN_MIN_HEIGHT, max_=GEN_MAX_HEIGHT)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=SELECTION_POOL_SIZE)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=GEN_MIN_HEIGHT, max_=GEN_MAX_HEIGHT)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=HEIGHT_LIMIT))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=HEIGHT_LIMIT))

stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)
pop = toolbox.population(n=POP_SIZE)
hof = tools.HallOfFame(HOF_SIZE)
pop, log = algorithms.eaMuPlusLambda(pop, toolbox,MU,LAMBDA, MATING_PROB, MUTATION_PROB, NUM_GENERATIONS, stats=mstats,halloffame=hof, verbose=True)


   	      	                                 fitness                                 	                      size                     
   	      	-------------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg     	gen	max     	min     	nevals	std      	avg    	gen	max	min	nevals	std    
0  	1024  	0.243311	0  	0.306841	0.176631	1024  	0.0296102	23.3184	0  	63 	4  	1024  	13.4102
1  	775   	0.205137	1  	0.251931	0.176631	775   	0.013728 	21.4658	1  	62 	3  	775   	12.4066
2  	817   	0.188245	2  	0.216094	0.176631	817   	0.00613293	19.8242	2  	62 	2  	817   	12.8652
3  	822   	0.181621	3  	0.19083 	0.175296	822   	0.00220317	18.7646	3  	59 	2  	822   	14.0695
4  	826   	0.179137	4  	0.184227	0.174102	826   	0.00167435	17.709 	4  	57 	3  	826   	9.73924
5  	829   	0.176728	5  	0.180392	0.173547	829   	0.00137555	14.5391	5  	57 	5  	829   	9.88244
6  	822   	0.174902	6  	0.177571	0.169682	822   	0.000962185	16.6992	6  

In [5]:
file=open('./evolved_funcs/best_funcs3','wb')
pickle.dump(hof,file)
file.close()


In [6]:
for hof_index in range(HOF_SIZE):
    nodes, edges, labels = gp.graph(hof[hof_index])
    print("Function ", hof[hof_index])
    test_type=['j30','j60','j90','j120']
    sum_total_dev=0
    sum_counts=0
    for typ in test_type:
        total_dev_percent,makespan,total_dev,count=statistics.evaluate_custom_rule(instance.instance,toolbox.compile(expr=hof[hof_index]),inst_type=typ,mode='parallel',option='forward')
        print(typ,total_dev_percent,makespan)
        log_file=open('results_log.txt','a+')
        log_file.write(str(hof[hof_index])+" : \n              "+INST_TYPE+"         "+typ+"         "+str(len(train_set))+"               "+str(NUM_GENERATIONS)+"          "+str(MATING_PROB)+"           "+str(MUTATION_PROB)+"         "+str(round(total_dev_percent,2))+"        "+str(makespan)+"       \n\n")
        log_file.close()
        sum_total_dev+=total_dev
        sum_counts+=count
    print("Aggregate % ",(sum_total_dev*100)/sum_counts)


    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw("./gp_trees/"+str(round(total_dev_percent,2))+"__3.png")


Function  add(mul(sub(LF, MaxRReq), add(LF, TSC)), max(LS, neg(add(LS, sub(ES, EF)))))
./j30/j309_9.sm (0.9459459459459459, 72) 18.638414821556573                                                              18.4217462387142                     (0.8085106382978723, 85) 18.02250123286325                     (0.11428571428571428, 39) 18.37913339518374                    
336 j30 files read
j30 18.638414821556573 20743
./j60/j609_9.sm (0.3875, 111) 17.445570547041534                                   14.00198929770458                     (0.13580246913580246, 92) 18.35297901009657                                         (0.0, 70) 17.520960748345                     (0.803921568627451, 92) 17.350401604598193                    
336 j60 files read
j60 17.445570547041534 28305
./j90/j909_9.sm (0.8, 135) 15.463648291450633                                        14.435373815345086                     (0.6883116883116883, 130) 15.574830625908378                    
480 j90 files read
j90 15.574